2.1 
Momentum indicators:
Ichimoku CLoud lines
Explanation:The Ichimoku Cloud (Ichimoku Kinko Hyo) is a five‑line trend‑following indicator that combines momentum and support‑resistance into a single overlay on price.There are 5 components of this indicator .For each component we use high,low,close and standard periods 9(tenkan),26(kijun),52(senkou).Periods 9, 26, 52 were chosen by Hosoda to match the Japanese trading calendar: roughly 1.5 weeks, 1 month, and 2 months of trading days.Their formulas are:
Conversion Line (Tenkan sen) = 9-PH+9-PL/2
Base Line (Kijun sen) = 26-PH + 26-PL/2
Leading Span A (Senkou span A) = CL + BL/2
Leading Span B (Senkou span B) = 52-PH + 52-PL/2
Lagging Span (Chikou Span) = Close Plotted 26 periods in the past
PH = Period High
PL = Period Low
BL = Base Line
CL = Conversion Line
Tenkan‑sen (conversion line) is the short‑term midpoint of the last 9 periods and measures fast momentum; when it crosses Kijun‑sen, a new move is often starting.
Kijun‑sen (base line) is the 26‑period midpoint and represents medium‑term equilibrium; price staying above it supports a bullish trend, below it supports a bearish trend. 
Senkou Span A and B are projected 26 periods into the future and form the cloud, projecting likely future support/resistance instead of only current levels. 
Chikou Span is the closing price shifted 26 periods back; when it stays above past price the bullish trend is confirmed, and below pastprice confirms bearish conditions.


In [18]:
import pandas as pd
def ichimoku(df: pd.DataFrame,high_col: str = "High",low_col: str = "Low",close_col: str = "Close",tenkan_period: int = 9,kijun_period: int = 26,
senkou_b_period: int = 52,displacement: int = 26) -> pd.DataFrame:
 high = df[high_col]
 low = df[low_col]
 close = df[close_col]
 tenkan_high = high.rolling(window=tenkan_period, min_periods=tenkan_period).max()
 tenkan_low  = low.rolling(window=tenkan_period, min_periods=tenkan_period).min()
 tenkan_sen = (tenkan_high + tenkan_low) / 2.0
 kijun_high = high.rolling(window=kijun_period, min_periods=kijun_period).max()
 kijun_low  = low.rolling(window=kijun_period, min_periods=kijun_period).min()
 kijun_sen = (kijun_high + kijun_low) / 2.0
 senkou_span_a = ((tenkan_sen + kijun_sen) / 2.0).shift(displacement)
 spanb_high = high.rolling(window=senkou_b_period, min_periods=senkou_b_period).max()
 spanb_low  = low.rolling(window=senkou_b_period, min_periods=senkou_b_period).min()
 senkou_span_b = ((spanb_high + spanb_low) / 2.0).shift(displacement)
 chikou_span = close.shift(-displacement)
 out = df.copy()
 out["tenkan_sen"] = tenkan_sen
 out["kijun_sen"] = kijun_sen
 out["senkou_span_a"] = senkou_span_a
 out["senkou_span_b"] = senkou_span_b
 out["chikou_span"] = chikou_span
 return out

2.Volume Indicators:Ease of Movement indicator:Explanation:The Ease of Movement(EOM) indicator is a volume based oscillator. It is designed to measure the relationship between price and volume and display that relationship as an oscillator that fluctuates between positive and negative values. The EOM fluctuates above and below a Zero Line. This is done in order to quantify the "ease" of price movements. A basic understanding is that when the EOM is in positive side, prices are advancing with relative ease. When the EOM is negative, prices are declining with relative ease.We calculate EOM in steps using Distance Moved and a Box Ratio that includes volume and the high–low range.If price moves a lot while volume is small, EOM becomes large,implies that price is moving easily and if price needs heavy volume to move, EOM is smaller implies more resistance.
Steps used to calculate EOM:
1. Calculate the distance moved.
((CurrentHigh + CurrentLow)/2 - (PreviousHigh + PreviousLow)/2) = Distance Moved
2. Calculate the Box Ratio which take volume and high/low range to produce the denominator in EOM calculations.
((CurrentVolume/100,000,000) / (CurrentHigh - CurrentLow))
3. Calculate a 1 Period EOM.
Distance Moved / Box Ratio = 1 Period EOM
4. Calculate a 14 Period EOM(commonly used)
14 Period Simple Moving Average of the 1 Period EOM = 14 Period EOM.
Length = 14 (smoothing period):
14 is the default and a 14‑period EOM is very common.Reason for choosing 14 is that it reduces the choppiness of the 1‑period EOM and gives a clearer trend/zero-line behavior for typical daily or swing analysis. 
Divisor (volume scaling):a Divisor input is needed to help produce a ratio number that’s not too big or too small and has higher-volume assets need a larger divisor so the indicator stays in single/double digits.Reason for choosing a divisor is that the raw volume values can be extremely large, so scaling makes EOM values readable and comparable across assets/timeframes (you adjust divisor based on typical volume).


In [19]:
import pandas as pd
import numpy as np
def eom(df: pd.DataFrame,high_col: str = "High",low_col: str = "Low",vol_col: str = "Volume",length: int = 14,divisor: float = 100000000.0
   )-> pd.DataFrame:
    high = df[high_col].astype(float)
    low  = df[low_col].astype(float)
    vol  = df[vol_col].astype(float)
    midpoint = (high + low) / 2.0
    distance_moved = midpoint - midpoint.shift(1)
    hl_range = (high - low).replace(0, np.nan)  #to avoid division by zero
    box_ratio = (vol / divisor) / hl_range
    eom_1p = distance_moved / box_ratio
    eom_sma = eom_1p.rolling(window=length, min_periods=length).mean()
    out = df.copy()
    out["eom_1p"] = eom_1p
    out["eom"] = eom_sma#sma==simple moving average
    return out

3.Volatility indicators:Keltner channels:Explanation:Keltner Channel refers to a technical analysis indicator composed of three separate lines. It includes a central moving average line along with channel lines located above and below the central one.It is named after American grain trader Chester W. Keltner, who described it in his 1960 book entitled “How to Make Money in Commodities.”.It was later revised, and the modern version uses the exponential moving average (EMA) of the price as the center line.The EMA is usually collected across 20 periods, contrasting to Keltner’s initial 10-day moving average.Keltner Channels use the Average True Range (ATR). This makes them smoother and less prone to the “whipsaw” effect of sudden price spikes, as ATR measures the absolute price range rather than variance from the mean.Formulae used:
Middle Line: A 20-period Exponential Moving Average (EMA) of the closing price.	
Band Width: Calculated using the 10-period Average True Range (ATR).
Upper/Lower Bands: The bands are placed 2.0 ATRs above and below the central EMA.
Upper Channel = EMA + (2 * ATR) 
Lower Channel = EMA - (2 * ATR) 
EMA Length (20): A 20-period Exponential Moving Average is the standard center line, providing a responsive measure of the current trend direction. 
ATR Length (10): The bands are set based on the Average True Range (ATR) over 10 periods. This captures recent volatility more effectively than a standard deviation (used in Bollinger Bands). 
Multiplier (2.0): The upper and lower bands are placed 2 ATRs away from the EMA. This width ensures the channel contains most price action (about 95%), making breakouts outside the channel significant events for trend trading.



In [20]:
import pandas as pd
def keltner_channels(df: pd.DataFrame, high_col: str = "High", low_col: str = "Low", close_col: str = "Close",ema_len: int = 20, 
atr_len: int = 10, multiplier: float = 2.0) -> pd.DataFrame:
    high = df[high_col].astype(float)
    low = df[low_col].astype(float)
    close = df[close_col].astype(float)
    basis = close.ewm(span=ema_len, adjust=False).mean() #Calculating basis of center line
    #Calculate ATR (Average True Range)
    prev_close = close.shift(1) 
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=atr_len, min_periods=atr_len).mean()
    #Calculate Bands
    upper = basis + (multiplier * atr)
    lower = basis - (multiplier * atr)
    out = df.copy()
    out['kc_basis'] = basis
    out['kc_upper'] = upper
    out['kc_lower'] = lower
    return out

Q2.2:Trading strategy using Ichimoku cloud lines,Ease of movement ,Keltner channels:The developed strategy is a trend following system with volatility confirmation designed to capture high momentum moves in Bitcoin while filtering out low-conviction noise. It operates on a hierarchical logic, starting with the Ichimoku cloud lines as a primary regime filter, by requiring price to be positioned above the cloud (Senkou Span A and B), the strategy ensures that capital is only deployed during structurally bullish phases, effectively sidestepping the bearish downtrends characteristic of 2022. Tactical entry is triggered by a volatility breakout above the Upper Keltner Channel, which signals that price is deviating significantly from its average range, indicating the start of an impulsive move. To prevent “fake-outs” (false breakouts), the Ease of Movement (EOM) indicator serves as a mandatory confirmation layer, a positive EOM ensures that the price advance is occurring with ease(efficient volume relative to price change), validating that the breakout has genuine market participation. Finally, the strategy utilizes a dynamic exit mechanism by closing positions when the price reverts below the Keltner Basis (central moving average), effectively acting as a trailing stop that protects capital once the specific momentum of the trend is exhausted.

In [21]:
import pandas as pd
import numpy as np
import yfinance as yf
#The above three indicators
def ichimoku(df: pd.DataFrame,high_col: str = "High",low_col: str = "Low",close_col: str = "Close",tenkan_period: int = 9,kijun_period: int = 26,
senkou_b_period: int = 52,displacement: int = 26) -> pd.DataFrame:
 high = df[high_col]
 low = df[low_col]
 close = df[close_col]
 tenkan_high = high.rolling(window=tenkan_period, min_periods=tenkan_period).max()
 tenkan_low  = low.rolling(window=tenkan_period, min_periods=tenkan_period).min()
 tenkan_sen = (tenkan_high + tenkan_low) / 2.0
 kijun_high = high.rolling(window=kijun_period, min_periods=kijun_period).max()
 kijun_low  = low.rolling(window=kijun_period, min_periods=kijun_period).min()
 kijun_sen = (kijun_high + kijun_low) / 2.0
 senkou_span_a = ((tenkan_sen + kijun_sen) / 2.0).shift(displacement)
 spanb_high = high.rolling(window=senkou_b_period, min_periods=senkou_b_period).max()
 spanb_low  = low.rolling(window=senkou_b_period, min_periods=senkou_b_period).min()
 senkou_span_b = ((spanb_high + spanb_low) / 2.0).shift(displacement)
 chikou_span = close.shift(-displacement)
 out = df.copy()
 out["tenkan_sen"] = tenkan_sen
 out["kijun_sen"] = kijun_sen
 out["senkou_span_a"] = senkou_span_a
 out["senkou_span_b"] = senkou_span_b
 out["chikou_span"] = chikou_span
 return out
def eom(df: pd.DataFrame,high_col: str = "High",low_col: str = "Low",vol_col: str = "Volume",length: int = 14,divisor: float = 100000000.0
   )-> pd.DataFrame:
    high = df[high_col].astype(float)
    low  = df[low_col].astype(float)
    vol  = df[vol_col].astype(float)
    midpoint = (high + low) / 2.0
    distance_moved = midpoint - midpoint.shift(1)
    hl_range = (high - low).replace(0, np.nan)  #to avoid division by zero
    box_ratio = (vol / divisor) / hl_range
    eom_1p = distance_moved / box_ratio
    eom_sma = eom_1p.rolling(window=length, min_periods=length).mean()
    out = df.copy()
    out["eom_1p"] = eom_1p
    out["eom"] = eom_sma#sma==simple moving average
    return out
import pandas as pd
def keltner_channels(df: pd.DataFrame, high_col: str = "High", low_col: str = "Low", close_col: str = "Close",ema_len: int = 20, 
atr_len: int = 10, multiplier: float = 2.0) -> pd.DataFrame:
    high = df[high_col].astype(float)
    low = df[low_col].astype(float)
    close = df[close_col].astype(float)
    basis = close.ewm(span=ema_len, adjust=False).mean() #Calculating basis of center line
    #Calculate ATR (Average True Range)
    prev_close = close.shift(1) 
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=atr_len, min_periods=atr_len).mean()
    #Calculate Bands
    upper = basis + (multiplier * atr)
    lower = basis - (multiplier * atr)
    out = df.copy()
    out['kc_basis'] = basis
    out['kc_upper'] = upper
    out['kc_lower'] = lower
    return out
def fetch_data(symbol="BTC-USD", start="2022-01-01", end="2025-01-01"):
    df = yf.download(symbol, start=start, end=end, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        try:
            df = df.xs(symbol, axis=1, level=1)
        except KeyError:
            df.columns = [col[0] for col in df.columns]
    df.columns = [c.capitalize() for c in df.columns]
    required_cols = ['High', 'Low', 'Close', 'Volume']
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Data is missing required columns: {missing}. Found: {df.columns.tolist()}")  
    return df
def generate_signals(df):
    df = df.copy()
    df['cloud_top'] = df[['senkou_span_a', 'senkou_span_b']].max(axis=1)
    c1_breakout = df['Close'] > df['kc_upper']
    c2_trend = df['Close'] > df['cloud_top']
    c3_volume = df['eom'] > 0
    c_exit = df['Close'] < df['kc_basis']
    signals = [0] * len(df)
    position = 0 
    start_index = 52 
    for i in range(start_index, len(df)):
        if position == 0:
            if c1_breakout.iloc[i] and c2_trend.iloc[i] and c3_volume.iloc[i]:
                signals[i] = 1 
                position = 1
        elif position == 1:
            if c_exit.iloc[i]:
                signals[i] = -1 
                position = 0           
    df['Signal'] = signals
    return df
def backtest_strategy(df, initial_capital=100000):
    cash = initial_capital
    holdings = 0
    portfolio_value = []
    trades = [] 
    for i in range(len(df)):
        price = df['Close'].iloc[i]
        signal = df['Signal'].iloc[i]
        date = df.index[i]
        if signal == 1 and cash > 0: # BUY
            holdings = cash / price
            cash = 0
            trades.append({'Date': date, 'Type': 'Buy', 'Price': price})   
        elif signal == -1 and holdings > 0: # SELL
            cash = holdings * price
            holdings = 0
            trades.append({'Date': date, 'Type': 'Sell', 'Price': price})
        current_val = cash + (holdings * price)
        portfolio_value.append(current_val)  
    df['Portfolio_Value'] = portfolio_value
    return df, pd.DataFrame(trades)
def calculate_metrics(df, trade_log):
    df['Daily_Return'] = df['Portfolio_Value'].pct_change()
    start_val = df['Portfolio_Value'].iloc[0]
    end_val = df['Portfolio_Value'].iloc[-1]
    net_profit = end_val - start_val
    total_return_pct = (net_profit / start_val) * 100
    risk_free_rate = 0.02 
    excess_returns = df['Daily_Return'] - (risk_free_rate/365)
    sharpe = np.sqrt(365) * (excess_returns.mean() / excess_returns.std())
    neg_returns = excess_returns[excess_returns < 0]
    sortino = np.sqrt(365) * (excess_returns.mean() / neg_returns.std())
    cum_returns = (1 + df['Daily_Return']).cumprod()
    peak = cum_returns.cummax()
    drawdown = (cum_returns - peak) / peak
    max_drawdown = drawdown.min() * 100
    wins = 0
    losses = 0
    if not trade_log.empty:
        sells = trade_log[trade_log['Type'] == 'Sell']
        buys = trade_log[trade_log['Type'] == 'Buy']
        min_len = min(len(buys), len(sells))
        for i in range(min_len):
            buy_price = buys.iloc[i]['Price']
            sell_price = sells.iloc[i]['Price']
            if sell_price > buy_price:
                wins += 1
            else:
                losses += 1

    print("-" * 40)
    print(f"STRATEGY PERFORMANCE (2022-2025)")
    print("-" * 40)
    print(f"Initial Capital:   ${start_val:,.2f}")
    print(f"Final Portfolio:   ${end_val:,.2f}")
    print(f"Net Profit:        ${net_profit:,.2f} ({total_return_pct:.2f}%)")
    print(f"Sharpe Ratio:      {sharpe:.2f}")
    print(f"Sortino Ratio:     {sortino:.2f}")
    print(f"Max Drawdown:      {max_drawdown:.2f}%")
    print(f"Winning Trades:    {wins}")
    print(f"Losing Trades:     {losses}")
    print("-" * 40)
if __name__ == "__main__":
    data = fetch_data()
    data = ichimoku(data)
    data = eom(data)
    data = keltner_channels(data)
    data = generate_signals(data)
    results_df, trades_df = backtest_strategy(data)
    calculate_metrics(results_df, trades_df)
    output_cols = ['Open', 'High', 'Low', 'Close', 'Signal', 'Portfolio_Value', 
                   'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b', 
                   'eom', 'kc_upper', 'kc_basis']
    final_cols = [c for c in output_cols if c in data.columns]
    data[final_cols].to_csv("BTC_Strategy_Q2_2_Results.csv")
    print("\nResults saved to 'BTC_Strategy_Q2_2_Results.csv'")


----------------------------------------
STRATEGY PERFORMANCE (2022-2025)
----------------------------------------
Initial Capital:   $100,000.00
Final Portfolio:   $209,226.82
Net Profit:        $109,226.82 (109.23%)
Sharpe Ratio:      1.00
Sortino Ratio:     1.61
Max Drawdown:      -23.12%
Winning Trades:    7
Losing Trades:     7
----------------------------------------

Results saved to 'BTC_Strategy_Q2_2_Results.csv'


/var/folders/59/0ztm7zcs51q3k63d6b4r7lsc0000gn/T/ipykernel_2061/796495162.py:66: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start, end=end, progress=False)
